## Lab 9: Integrating ArcGIS Pro with Python  

In this Lab, we will introduce how to invoke the ArcGIS geoprocessing functions in the ArcPy package to run a set of spatial statistics methods we introduced this semester, including the spatial clustering analysis, OLS and GWR regressions, and the spatial interpolation methods. ArcPy also facilitates geoprocessing efficiency and allows for batch processing with hundreds of shapefiles as well. 

Through this week, you will use the ArcPy package to: 
- (1) understand how to invoke the appropriate ArcPy functions by reading the tool reference;
- (2) run the ESDA (Exploratory Spatial Data Analysis) tools, such as the Global Moran's I index, Local Moran's I index, and the hot spot analysis;
- (3) run the OLS and GWR regressions;
- (4) run the spatial interpolation functions;
- (5) automate your geoprocessing tools using for-loop and ArcPy.

### 1. Setting geoprocessing environments

In [1]:
# set geoprocessing environments


### 2. Clustering Analysis


Recall that ArcGIS provides statistical cluster analysis tools that you can use to minimize the subjectivity in your maps. The Hot Spot Analysis and Cluster and Outlier Analysis tools both use statistics to detect spatial patterns in your data, but they provide slightly different information about these patterns.

- Hot Spot Analysis tools use the Getis-Ord Gi* statistic to identify spatial clusters of statistically significant clusters of high values (hot spots) and low values (cold spots).

- Cluster and Outlier Analysis tools use the Anselin Local Moran's I statistic to identify statistically significant hot spots, cold spots, and spatial outliers. Spatial outliers are significantly dissimilar from their neighboring features.

We will use the socialmedia dataset to identify the clusters of counties with high social media usage. 

#### Now, Let us first run the Global Moran's I: 

In [ ]:
# Global Moran'I

#### Local Moran's I (Cluster and outlier analysis)

arcpy.stats.ClustersOutliers(Input_Feature_Class, Input_Field, Output_Feature_Class, Conceptualization_of_Spatial_Relationships, Distance_Method, Standardization, {Distance_Band_or_Threshold_Distance}, {Weights_Matrix_File}, {Apply_False_Discovery_Rate__FDR__Correction}, {Number_of_Permutations}, {number_of_neighbors})

In [8]:
Input_Feature_Class = "/Lab9_data/socialmedia_northeast/socialmedia_ne.shp"
Input_Field = "Pct_Facebo"
Output_Feature_Class = "/output/somedia_local.shp"
Spatial_matrix = "CONTIGUITY_EDGES_CORNERS"
Distance_Method = "EUCLIDEAN_DISTANCE"
Standardization = "ROW"
FDR = "NO_FDR"
Permutations = 499

arcpy.ClustersOutliers_stats(Input_Feature_Class, Input_Field, 
                             Output_Feature_Class,
                             Spatial_matrix, Distance_Method, 
                             Standardization, "#", "#", FDR, Permutations)

# arcpy.ClustersOutliers_stats("socialmedia/County_socialmedia.shp", "Pct_Facebo", 
#                              "socialmedia/LISA_output.shp",
#                              "CONTIGUITY_EDGES_CORNERS","EUCLIDEAN_DISTANCE", 
#                              "ROW", "#", "#", "NO_FDR", 499)

id,value
0,C:\Users\Lwz12\Desktop\Lab9\\output\somedia_local.shp
1,LMiIndex
2,LMiZScore
3,LMiPValue
4,COType
5,FID


#### Hot Spot Analysis

In [14]:
arcpy.HotSpots_stats("/socialmedia_northeast/socialmedia_ne.shp", "Pct_Facebo", "/Lab9_data_output/somedia_HS.shp",
                     "CONTIGUITY_EDGES_CORNERS", "EUCLIDEAN_DISTANCE", 
                     "ROW", "#", "#", "#", "NO_FDR")

id,value
0,D:\TA_CRP5680_UrbanDataSci\Lab9\Lab9_data_demo\\Lab9_data_output\somedia_HS.shp
1,GiZScore
2,GiPValue
3,FID


### 3. Regressions using ArcGIS Pro Python

Let us now try to run the OLS and GWR regressions using Arcpy built-in functions and the Beijing housing dataset. The regressions require the dataset to be in specific format and therefore, require data processing before the regression analysis. 
- We can read the dataset (in .shp) as a Spatially Enabled DataFrame (SEDF) by invoking the function called `pd.DataFrame.spatial.from_featureclass()`. The function read a `.shapefile` into ArcGIS Python and convert it to a spatial Dataframe. 

- Similarly, we can use the `.spatial.to_featureclass` to convert the spatially Enabled DataFrame (SEDF) to shapefile and save it in a file folder. 

- See more about SEDF at https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/

Now we read the Beijing resale housing price dataset (.shp) into Python, categorizes the data into two parts -- houisng sales in urban core area and housing sales in the sururbs. Then we run OLS for each of them separately by using for-loop. 

#### 3.1 OLS

In [10]:
#### Categorize the data into two parts: the urban core and the suburban area.
urban_core_lst = ["Dongcheng", "Xicheng", "Chaoyang", "Haidian", "Fengtai", "Shijingshan"]

gdf_core = gdf.loc[(gdf["disname_en"].isin(urban_core_lst)), :]
gdf_core["hid"] = gdf["FID"]

gdf_sub = gdf.loc[~(gdf["disname_en"].isin(urban_core_lst)), :]
gdf_sub["hid"] = gdf["FID"]

In [16]:
gdf_core.spatial.to_featureclass(workspace + "/output/housing_core.shp")
gdf_sub.spatial.to_featureclass(workspace + "/output/housing_sub.shp")

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


'C:\\Users\\Lwz12\\Desktop\\Lab9\\output\\housing_sub.shp'

## 3.2 GWR

Let us run a GWR regression using Arcpy and the whole housing dataset.

In [33]:
arcpy.stats.GWR("/housing/Beijing_housing2016.shp", "unitprice", "CONTINUOUS", "size;totfloor;dist2sub", 
     "/Lab9_data_output/result_GWR", "DISTANCE_BAND", "GOLDEN_SEARCH")

id,value
0,D:\TA_CRP5680_UrbanDataSci\Lab9\Lab9_data_demo\\Lab9_data_output\result_GWR.shp
1,
2,


## 4. Spatial Interpolation

Arcpy also incorporates the spatial interpolation tools in the Spatial Analyst Module of ArcGIS which provides four commonly-used interpolation methods -- IDW, Kriging, Nearest Neighbor, and Spline. 
- We will next use the Beijing monthly PM2.5 concentration in 2014 as an example to explore how the functions work.     

In [34]:
gdf_air = pd.DataFrame.spatial.from_featureclass(workspace + "/airquality/gdf_2014.shp")
gdf_air.head(3)

,FID,Eng_name,lon,lat,201401,201402,201403,201404,201405,201406,201407,201408,201409,201410,201411,201412,SHAPE
0,0,Dongsi,116.417,39.929,103.989944,154.159818,99.812829,92.293601,64.824013,58.256827,99.256031,69.889981,68.090435,117.270192,82.254303,56.525662,"{""x"": 116.417, ""y"": 39.929, ""spatialReference""..."
1,1,Tiantan,116.407,39.886,98.790410,144.743320,93.674694,88.977833,64.030937,61.215977,92.117828,68.384782,68.613844,115.402447,89.332727,68.863611,"{""x"": 116.407, ""y"": 39.886, ""spatialReference""..."
2,2,Guanyuan,116.339,39.929,101.438401,150.010663,94.243136,95.021284,63.891839,60.105481,88.617794,73.023634,70.812389,130.514968,90.295243,62.054568,"{""x"": 116.339, ""y"": 39.929, ""spatialReference""..."


In [35]:
# create the average PM2.5 concentrations by seasons


In [37]:
gdf_air.spatial.to_featureclass(workspace + "/Lab9_data_output/airquality2014.shp")

'D:\\TA_CRP5680_UrbanDataSci\\Lab9\\Lab9_data_demo\\Lab9_data_output\\airquality2014.shp'

In [38]:
# IDW using for-loop
gdf_air.spatial.to_featureclass(workspace + "/Lab9_data_output/airquality2014.shp")
for i in ["spring", "summer", "fall", "winter"]:
    # Set local variables
    inPointFeatures = workspace + "/Lab9_data_output/airquality2014.shp"
    zField = i
    outLayer = "#"
    outRaster = workspace + "/Lab9_data_output/idw_" + str(i) + ".tif"
    cellSize = "#"
    power = 2

    # Execute IDW
    arcpy.IDW_ga(inPointFeatures, zField, outLayer, outRaster, cellSize, 
                 power)


### 5. A Brief Introduction of ArcGIS API Python

- For some of you who are interested in using the ArcGIS Online and the ArcGIS API for Python, Here is a small example of exploring the geolocations of Ithaca restaurants by combing ArcGIS Online and Python. For more about the package of the ArcGIS API for Python, see this website: https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html  

In [15]:
from arcgis.gis import GIS
gis = GIS() # Anonymous Login to ArcGIS Online

In [83]:
# initialize your map
map = gis.map("Ithaca, New York")
map

MapView(layout=Layout(height='400px', width='100%'))

In [84]:
# set parameters for your map
map.center = [42.43983599374521, -76.49733885932113]
map.zoom = 16
map.basemaps

['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oceans', 'osm', 'satellite', 'streets', 'streets-navigation-vector', 'streets-night-vector', 'streets-relief-vector', 'streets-vector', 'terrain', 'topo', 'topo-vector']

In [64]:
# change the basemap
map.basemap = 'streets'

In [142]:
# search the layer you want through ArcGIS Online
rest = gis.content.search("Ithaca eat", 
                          item_type = "Feature Layer", max_items = 5)
rest

[<Item title:"whereEatLocal" type:Feature Layer Collection owner:svetla.borovska_tompkinscounty>, <Item title:"whereEat" type:Feature Layer Collection owner:svetla.borovska_tompkinscounty>]

In [143]:
from IPython.display import display
for item in rest:
    display(item)

<Item title:"whereEatLocal" type:Feature Layer Collection owner:svetla.borovska_tompkinscounty>

<Item title:"whereEat" type:Feature Layer Collection owner:svetla.borovska_tompkinscounty>

In [145]:
rest = rest[1]
map.add_layer(rest)

In [139]:
for lyr in item.layers:
    print(lyr.properties.name)

whereEat


In [136]:
data = item.layers[0]

In [137]:
sdf = pd.DataFrame.spatial.from_layer(data)

In [138]:
sdf

,FID,Image_URL,Website,Short_Desc,Title,Number,Desc1,SHAPE
0,1,http://geo.tompkins-co.org/images/EatStayGo/Un...,http://www.unwindcafe.com/,40 Catherwood Rd,Unwind Coffee House and Internet Cafe,40,Awesome coffee shop in The Shops at Ithaca Mall,"{""x"": 845236.0435710847, ""y"": 905142.401368916..."
1,2,http://geo.tompkins-co.org/images/EatStayGo/Gi...,https://www.gimmecoffee.com/,506 West State St,Gimme! Coffee,30,"Espresso, drip brews & small-batch roasted cof...","{""x"": 840951.7739244178, ""y"": 888804.418296918..."
2,3,http://geo.tompkins-co.org/images/EatStayGo/Gi...,https://www.gimmecoffee.com/,430 N Cayuga St,Gimme! Coffee,34,"Espresso, drip brews & small-batch roasted cof...","{""x"": 842848.8364032507, ""y"": 890626.225676916..."
3,4,http://geo.tompkins-co.org/images/EatStayGo/Th...,http://www.theheightsithaca.com/,903 Hanshaw Rd,The Heights,23,Refined spot with patio seating featuring Medi...,"{""x"": 848126.1467147507, ""y"": 899538.437183171..."
4,5,http://geo.tompkins-co.org/images/EatStayGo/Ci...,http://www.ciaoithaca.com/index.html,2 Hickory Hollow Ln,Ciao!,10,Stylish yet casual Italian resturant,"{""x"": 845631.8488818333, ""y"": 906392.161008499..."
...,...,...,...,...,...,...,...,...
126,127,https://static1.squarespace.com/static/58e2611...,https://www.liquidstatebeer.com/,620 W Green St,Licuid State Brewing Co,131,A craft brewery and beer hall,"{""x"": 840621.1289164163, ""y"": 888516.532061666..."
127,128,http://www.boleatery.com/img/bol-new-photo.jpg,http://www.boleatery.com/,222 E State St,BoL,132,Ramen Restaurant,"{""x"": 843794.4260475002, ""y"": 888904.631351083..."
128,129,http://geo.tompkins-co.org/images/EatStayGo/Ma...,https://www.maru-ramen.com/,512 W State St,Maru Ramen,133,Ramen Restaurant,"{""x"": 840863.2990357503, ""y"": 888797.179466255..."
129,130,http://geo.tompkins-co.org/images/EatStayGo/Fr...,https://www.facebook.com/francoandsalvosithacany/,508 W State St,Franco's Pizzeria,134,Pizza Restaurant,"{""x"": 840926.4130827524, ""y"": 888797.737207919..."


### 6. Homework
#### Due: April 20th; Total points: 50 pts

### Question 1 (15 pts)
Tornadoes occur in many parts of the world, but they are found most frequently in the United States. Tornadoes are the most violent of all atmospheric storms and in the United States alone have caused an average of more than 80 deaths and 1,400 injuries each year (based on 1950–2008 data). Tornado formation is complex and no two tornadoes are the same; however, they need certain conditions to form, including intense or unseasonable heat. Wind speed within a tornado can vary from just above 0 mph up to 70 mph, with an average of 30 mph (NOAA). The Fujita damage scale is used to quantify the intensity of a tornado.

Using tornado location data from the United States severe weather report database, provided by the National Oceanic and Atmospheric Administration (NOAA)/National Weather Service Storm Prediction Center (https://www.spc.noaa.gov/gis/svrgis/), you can find the detailed infromation about the Tornades. The dataset contains all records of Tornadoes in the United States from 2008 to 2011. Please use the dataset to calculate and visualize the following tasks based the the Fujita damage scale (FSCALE): 
- (1) the Global Moran's I index
- (2) the Local Moran's I index
- (3) the Hot spot analysis

### Question 2 (15 pts)
Run OLS and GWR regressions using the Beijing Housing dataset ("Beijing_housing1416.shp"). 
- (1) Categorize the housing dataset into two parts: the urban core and the suburb. The urban core includes six districts (see field "disname_en"): ["Dongcheng", "Xicheng", "Chaoyang", "Haidian", "Fengtai", "Shijingshan"]. The suburbs are the rest of the districts. 
- (2) Run OLS regressions for both the urban core and the suburbs using for-loop in Python. Show your major regression results. 
- (3) Run a GWR regressions for the whole dataset. Show your major regression results. 


### Question 3 (20 pts):
We will calculate the annual average PM2.5 concentration and obtain the spatial interpolation surfaces using the Beijing PM2.5 dataset from 2014 to 2018 (PM25_1418.shp).
- (1) Calculate the annual average PM2.5 for each year (2014-2018) and attach the result to your current dataframe (hint: see Section 4 for help).
- (2) Create spatial interpolation surface for each year and visualize them using the same legend and scale for comparison purposes.  
